In [1]:
from urllib import urlopen
import time
import datetime
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from itertools import izip
import pandas as pd
import cPickle
import re
import csv
import dill
from config import *
driver = webdriver.Firefox(firefox_profile = profile, executable_path= webdriver_path)


In [67]:
urls = ['https://www.iens.nl/restaurants-zuid-holland/rt2132'
,'https://www.iens.nl/restaurants-zeeland/rt2131'
,'https://www.iens.nl/restaurants-utrecht/rt2130'
,'https://www.iens.nl/restaurants-overijssel/rt2129'
,'https://www.iens.nl/restaurants-noord-holland/rt2128'
,'https://www.iens.nl/restaurants-noord-brabant/rt2127'
,'https://www.iens.nl/restaurants-limburg/rt2126'
,'https://www.iens.nl/restaurants-groningen/rt2125'
,'https://www.iens.nl/restaurants-gelderland/rt2124'
,'https://www.iens.nl/restaurants-friesland/rt2123'
,'https://www.iens.nl/restaurants-flevoland/rt2122'
,'https://www.iens.nl/restaurants-drenthe/rt2121']

today = datetime.date.today().strftime('%Y%m%d')
df = pd.DataFrame()

In [74]:
for url in urls[5:]:
    # go to page
    driver.get(url)

    # get number of pages
    for i in xrange(1,200):
        try:
            max_page = int(driver.find_element_by_xpath("//.[@data-page-number='%d']" % i).text)
        except:
            1
    print(url, str(max_page))
    
    for pageNo in range(max_page - 1):
        driver.get(url + '#page='+str(pageNo+1))
        time.sleep(5)
        
       
        for resto_object in driver.find_elements_by_class_name('resultItem'):

            names = resto_object.find_elements_by_xpath(".//h3[@class='resultItem-name']/a")
            if len(names) > 0:
                name = names[0].text.encode("ascii","ignore")
            else: 
                name = ''

            keukens = resto_object.find_elements_by_xpath(".//span[@class='restaurantTag restaurantTag--medium']")
            if len(keukens) > 0:
                keuken = keukens[0].text.encode("ascii","ignore")
            else: 
                keuken = ''


            plaatsen = resto_object.find_elements_by_xpath(".//div[@class='resultItem-address']")
            if len(plaatsen) > 0:
                plaats = plaatsen[0].text.encode("ascii","ignore")
            else: 
                plaats = ''    

            prijzen = resto_object.find_elements_by_xpath(".//div[@class='resultItem-averagePrice']")
            if len(prijzen) > 0:
                prijs = prijzen[0].text.encode("ascii","ignore")
            else: 
                prijs = ''       

            cijfers = resto_object.find_elements_by_xpath(".//span[@class='rating-ratingValue']")
            if len(cijfers) > 0:
                cijfer = cijfers[0].text.encode("ascii","ignore")
            else: 
                cijfer = ''     

            reviews = resto_object.find_elements_by_xpath(".//div[@class='reviewsCount reviewsCount--small']/a")
            if len(reviews) > 0:
                review = reviews[0].text.encode("ascii","ignore")
            else: 
                review = ''   

            links = resto_object.find_elements_by_xpath(".//h3[@class='resultItem-name']/a")
            if len(links) > 0:
                link = links[0].get_attribute("href")
                id = resto_object.get_attribute('data-restaurant-id')
            else: 
                link = ''     	

            # union
            restoInfo = pd.DataFrame({'id' : id,
                            'naam': name,
                              'plaats':plaats,
                              'cijfer':cijfer,
                              'prijs':prijs,
                              'keuken':keuken,
                              'reviews':review,
                              'url':link}, index=[id])

            df = df.append(restoInfo, ignore_index=True)

('https://www.iens.nl/restaurants-noord-brabant/rt2127', '97')
('https://www.iens.nl/restaurants-limburg/rt2126', '63')
('https://www.iens.nl/restaurants-groningen/rt2125', '24')
('https://www.iens.nl/restaurants-gelderland/rt2124', '81')
('https://www.iens.nl/restaurants-friesland/rt2123', '31')
('https://www.iens.nl/restaurants-flevoland/rt2122', '10')
('https://www.iens.nl/restaurants-drenthe/rt2121', '21')


In [75]:
print len(df.drop_duplicates()), len(df)

19597 24375


In [76]:
df = df.drop_duplicates()

In [81]:
import cPickle as pickle
with open(today+'_resto_urls_to_scrape.cpickle','wb') as f:
    pickle.dump(df, f)

In [77]:
# save
df.to_csv(today + '_resto_urls_to_scrape.csv')

In [78]:
today

'20180403'